# Deployment

A Deployment provides declarative updates for Pods and ReplicaSets.

You describe your desired state in a Deployment, and the Deployment Controller's job is to make the current state match the desired state. You declare your hopes and dreams, and it's Kubernetes' job to make them come true.

### Why Deleting a Pod Doesn't Feel Like a Deletion

Take a look at the YAML file for your current deployment in the CLI:

```python
kubectl get deployment synergychat-web -o yaml
```

Edit the deployment and change the number of replicas from 2 to 10:

```python
kubectl edit deployment synergychat-web
```

Keep using `kubectl get pod` to check on your pods until all 10 are in a "ready" state. Once they are, run:

```python
kubectl proxy
```

# Replica Sets

A [ReplicaSet](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/) maintains a stable set of replica Pods running at any given time. It's the thing that makes sure that the number of Pods you want running is the same as the number of Pods that are actually running.

You might be thinking, "I thought that's what a Deployment does." Well...yes.

A Deployment is a higher-level abstraction that manages the ReplicaSets for you. You can think of a Deployment as a wrapper around a ReplicaSet. Here's the rub: 

*You will probably never use ReplicaSets directly. I just need to mention what they are because you'll hear the term thrown around, and might even see them referenced in logs and such.*

Let's take a look at the ReplicaSets that are running in your cluster:

```python
kubectl get replicasets
```

Just like with pods, notice that we never directly created the replica set. We created a deployment, and the deployment created the replica set.

# Config

Kubernetes resources are primarily configured using YAML files. We've used the `kubectl edit` command to edit resources in the cluster on-demand.

Download a copy of your deployment's YAML file and save it in your current directory:

```python
kubectl get deployment synergychat-web -o yaml > web-deployment.yaml
```

Then open it in your text editor. There are 5 top-level fields in the file:
- `apiVersion: apps/v1` - Specifies the version of the Kubernetes API you're using to create the object (e.g., apps/v1 for Deployments).
- `kind`: Deployment - Specifies the type of object you're configuring
- `metadata` - Metadata about the deployment, like when it was created, its name, and its ID
- `spec` - The desired state of the deployment. Most impactful edits, like how many replicas you want, will be made here.
- `status` - The current state of the deployment. You won't edit this directly, it's just for you to see what's going on with your deployment.

You can freely edited anything in the yaml file then apply it back with:

```python
kubectl apply -f web-deployment.yaml
```

You should get a warning that lets you know that you're missing the `last-applied-configuration` annotation. That's okay! we got that warning because we created this deployment the quick and dirty way, by using kubectl create deployment instead of creating a YAML file and using `kubectl apply -f`.

Apply the configuration a second time, you won't get the warning.

If you're familiar with Vim, you can directly edited it on terminal with:

```python
kubectl edit deployment synergychat-web
```

# API Service

We've deployed one service, and we've deployed multiple instances of it. Time to deploy a second service!

This service doesn't serve a webpage! It's a JSON API. It's the backend for our chat application. By deploying the API and configuring the front-end to talk to it, we'll have a functional chat application!

Let's write one from scratch:

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: synergychat-api
  labels:
    app: synergychat-api
spec:
  replicas: 1
  selector:
    matchLabels:
      app: synergychat-api
  template:
    metadata:
      labels:
        app: synergychat-api
    spec:
      containers:
        - name: synergychat-api
          image: ghcr.io/synergychat/api:latest
```

Apply it then check with proxy:

```python
kubectl apply -f api-deployment.yaml
kubectl proxy
```

# Thrashing Pods

One of the most common problems you'll run into when working with Kubernetes is Pods that keep crashing and restarting. This is called ***"thrashing"*** and it's usually caused by one of a few things:
- The application recently had a bug introduced in the latest image version
- The application is misconfigured and can't start properly
- A dependency of the application is misconfigured and the application can't start properly
- The application is trying to use too much memory and is being killed by Kubernetes


### What is "CrashLoopBackoff"?

When a pod's status is `CrashLoopBackoff`, that means the container is crashing (the program is exiting with error code `1`).

Because Kubernetes is all about building self-healing systems, it will automatically restart the container. However, each time it tries to restart the container, if it crashes again, it will wait longer and longer in between restarts. That's why it's called a "backoff".

To fix a thrashing pod, you need to find out why it's crashing.